In [27]:
import pandas as pd
from konlpy.tag import Twitter
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from collections import Counter
import nltk

srlist = pd.read_csv('raw_data/test_data', sep=',', encoding='utf-8')

In [28]:
srlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18422 entries, 0 to 18421
Data columns (total 23 columns):
서비스요청접수월          18422 non-null int64
서비스요청접수일자         18422 non-null object
SR접수시간            18422 non-null int64
SR접수시간대(60분)      18422 non-null int64
서비스유형_NEW대분류코드    18422 non-null object
서비스유형_NEW대분류      18422 non-null object
서비스유형_NEW중분류코드    18422 non-null object
서비스유형_NEW중분류      18422 non-null object
서비스유형_NEW소분류코드    18422 non-null object
서비스유형_NEW소분류      18422 non-null object
인입채널 (SR)         18422 non-null object
서비스요청번호           18422 non-null object
상품코드              18422 non-null int64
상품명               16564 non-null object
고객번호              18422 non-null int64
서비스요청접수건수         18422 non-null int64
접수시간              18413 non-null object
주문상태              18413 non-null object
문의유형              18129 non-null object
내용                18128 non-null object
Unnamed: 20       2416 non-null object
Unnamed: 21       1 non-null object
Unnamed: 22       2 no

# DataFrame에 필요한 내용 집어넣기

In [76]:
df_sr = pd.DataFrame(srlist, columns=["서비스요청번호","내용"])
df_sr_filter = df_sr.dropna()

In [80]:
len(df_sr_filter)

18128

In [30]:
srcontents = srlist['내용']

# 한글 명사 추출을 위한 모듈 로드

In [31]:
t = Twitter()
h = Hannanum()
k = Kkma()

# SR내용만 DataFrame에 따로 넣기

In [32]:
df_srcontents = srcontents.to_frame()

In [33]:
type(df_srcontents)

pandas.core.frame.DataFrame

# DataFrame을 List로 형변환
- Twitter, Hannanum 모듈에 값을 넣으려면 List형으로 변환해야 함

In [34]:
arrTemp = df_srcontents.values.tolist()

In [35]:
tokens_ko = t.morphs(arrTemp[0][0])

In [36]:
ko = nltk.Text(tokens_ko, name='SR텍스트')
ko.vocab()

t.pos(arrTemp[0][0])

[('왜', 'Noun'),
 ('싱글', 'Noun'),
 ('만', 'Josa'),
 ('배송', 'Noun'),
 ('되었', 'Verb'),
 ('나요', 'Eomi'),
 ('?', 'Punctuation'),
 ('퀸', 'Noun'),
 ('은', 'Josa'),
 ('언제', 'Noun'),
 ('오는', 'Verb'),
 ('지요', 'Eomi'),
 ('..', 'Punctuation'),
 ('배송', 'Noun'),
 ('완료', 'Noun'),
 ('되', 'Verb'),
 ('지', 'Eomi'),
 ('않았', 'Verb'),
 ('는데', 'Eomi'),
 ('..', 'Punctuation'),
 ('확인', 'Noun'),
 ('부탁드립니', 'Adjective'),
 ('다', 'Eomi'),
 ('.', 'Punctuation')]

In [37]:
arrTemp[0][0]

'왜 싱글만 배송되었나요? 퀸은 언제오는지요.. 배송완료되지 않았는데.. 확인부탁드립니다.'

In [38]:
nouns = t.nouns(arrTemp[0][0])

In [39]:
count = Counter(nouns)
count

Counter({'배송': 2, '싱글': 1, '언제': 1, '완료': 1, '왜': 1, '퀸': 1, '확인': 1})

In [40]:
for letter in count:
    print(letter, count[letter])

완료 1
확인 1
배송 2
퀸 1
왜 1
언제 1
싱글 1


In [41]:
arrTempRow = []
for letter in count:
    arrTempRow.append(letter + " " + str(count[letter]))

In [42]:
arrTempRow

['완료 1', '확인 1', '배송 2', '퀸 1', '왜 1', '언제 1', '싱글 1']

In [43]:
for n, c in count.most_common(5):
    print(n,c)

배송 2
완료 1
확인 1
퀸 1
왜 1


In [44]:
len(arrTemp)

18422

# 컨텐츠 비어있는 SR 1차 정제 

In [45]:
arrContents = []

In [46]:
for row in arrTemp:
    if type(row[0]) == str:
        arrContents.append(row[0])

In [47]:
len(arrContents)

18128

# SR건별로 WordCount

In [70]:
l = []
l.append('foo')
l.append('bar')
l.append('baz')

s = ' '.join(l)
s

'foo bar baz'

In [71]:
arrTotal = []
for row in arrContents:
    # arrRow = []
    row_nouns = t.nouns(row)
    row_counter = Counter(row_nouns)
    arrTempRow = []
    
    for letter in row_counter:
        arrTempRow.append(letter + " " + str(row_counter[letter]))
    completedRow = ' '.join(arrTempRow)
    arrTotal.append(completedRow)

In [72]:
len(arrTotal)

18128

In [73]:
for value in arrTotal:
    print(value)

완료 1 확인 1 배송 2 퀸 1 왜 1 언제 1 싱글 1
데 1 바로 1 결제 1 눌럿 1 안눌려 1 취소 2 요청 1
연락 1 한번 1 취소 2
현금영수증 1 발급 1
적립 2 완료 1 혹시 1 저 1 된거 1 결제 1 얼마 1 예상 1 딜로 2 언제 1 매직 2 상품 2
데 1 고 1 달라 1 동생 1 다른 1 매장 1 취소 2 주문 3 오늘 1 햇 1 사 1 삿 1 제품 1 거 1
뭉침현상 1 재질 1 상품 1
카드 1 정보 1 할인 1 행사 1 답변 1 월달 1 알 1 부탁 1
카드 1 프로 1 지금 1 결제 1 국민카드 1 변경 2 수 1 혜택 1
포장 2 그대로 1 주문 1 박스 1 교환 1 그레이 1 월 1 오늘 2 비닐 1 확인 1 못 1 일 1 색상 1 사진 1
건데 1 이벤트 1 상품 1 방송 1 구입 1 쿠폰 1
날 1 데 1 은 1 송현동 1 부산 1 언제 1 벌써 1 도착 1 햇 1 파크빌 1 달서구 1 물 1 와 1 뉴 1
바지 1 상품권 1 문화 1
률좀 1 카드 2 오늘 1 내일 1 할인 3 며칠 1 사 1 동안 1 국민카드 1 데 1 날짜 1
구매 1 원 1 후 1 해결 1 너무 1 교환 1 사이즈 2 물건 1 실 1 재고 1 방법 1
주문 1 휴일 1 번호 1 해도 1 진행 1 취소 2 요 1
구매 1 주문 1 니 1 개입 1 후 1 전 1 처리 1 연락 1 분 1
결재 1 다른 1 저 1 박스 1 때 1 만약 1 사과 1 변 1 사과주스 1 추첨 1 언제 1 관련 1 결제 1 수단 1 경하 1 방송 1 당첨 1 문의 1 카드 1 배송 1 문자 1
언제 1 장사 1 제품 1 미궁 1
결재 1 프로 2 앱 1 신한 1 월 1 시 1 변경 1 청구 2 주문 1 할인 1 일 1 국민카드 1
연락 1 배 1 송전 1
오라 1 구매 1 시나 1 거의 1 혹 1 검정 1 너무 1 신발 1 신고 1 블랙 1 학교 1 네이비 1 컬러 1 시간 1 그 1 상품 1 사진 1 설명 1 알 1 문의 1
카드 1 금액 1 할인 2 계좌 1 가요 1 

In [89]:
df1 = pd.DataFrame(arrTotal, columns=['contents'])
df1['contents']

0                         완료 1 확인 1 배송 2 퀸 1 왜 1 언제 1 싱글 1
1                       데 1 바로 1 결제 1 눌럿 1 안눌려 1 취소 2 요청 1
2                                           연락 1 한번 1 취소 2
3                                             현금영수증 1 발급 1
4        적립 2 완료 1 혹시 1 저 1 된거 1 결제 1 얼마 1 예상 1 딜로 2 언제...
5        데 1 고 1 달라 1 동생 1 다른 1 매장 1 취소 2 주문 3 오늘 1 햇 1...
6                                         뭉침현상 1 재질 1 상품 1
7                   카드 1 정보 1 할인 1 행사 1 답변 1 월달 1 알 1 부탁 1
8                 카드 1 프로 1 지금 1 결제 1 국민카드 1 변경 2 수 1 혜택 1
9        포장 2 그대로 1 주문 1 박스 1 교환 1 그레이 1 월 1 오늘 2 비닐 1 ...
10                          건데 1 이벤트 1 상품 1 방송 1 구입 1 쿠폰 1
11       날 1 데 1 은 1 송현동 1 부산 1 언제 1 벌써 1 도착 1 햇 1 파크빌 ...
12                                         바지 1 상품권 1 문화 1
13       률좀 1 카드 2 오늘 1 내일 1 할인 3 며칠 1 사 1 동안 1 국민카드 1 ...
14       구매 1 원 1 후 1 해결 1 너무 1 교환 1 사이즈 2 물건 1 실 1 재고 ...
15                       주문 1 휴일 1 번호 1 해도 1 진행 1 취소 2 요 1
16                구매 1 주문 1 니 1 개입 1 후 1 전 1 처리 1 연락 1 분

# 기존 SR 데이터에 WordCount 결과 더하기

In [90]:
pd.concat([df_sr_filter, df1])

,contents,내용,서비스요청번호
0,NaN,왜 싱글만 배송되었나요? 퀸은 언제오는지요.. 배송완료되지 않았는데.. 확인부탁드립니다.,S090137794
1,NaN,취소하고싶은데취소가 안되내요 결제하고바로취소하려고눌럿는데 안눌려요 취소요청부탁드려요,S090137796
2,NaN,연락이 한번도 없네요 ㅠㅠ 취소부탁드립니다취소 가능한가요?,S090137799
3,NaN,현금영수증010 4506 8361발급해주세요~,S090137808
4,NaN,저 이상품 매직딜로 결제 완료 된거맞는거죠? 예상 적립은 얼마이며 언제 적립되나요?...,S090137820
5,NaN,이거 동생이 사달라고 해서 주문햇는데오늘 매장서 다른제품삿다고 주문취소해달라고 하네...,S090137842
6,NaN,상품재질이 으깨어지면서 뭉침현상이 있습니다,S090137850
7,NaN,"1월달 카드할인행사 정보를 알고싶은데요,~답변 부탁 드립니다",S090137858
8,NaN,결제카드 변경 하려는데 지금 국민카드 변경하면 7프로 혜택을 받을 수 있나요?,S090137871
9,NaN,12월 16일 주문했는데 오늘 포장을 뜯어서 확인을 했어요..그레이로 주문을 했는데...,S090137865


# 전체 SR 병합

In [ ]:
tempContent = ' '.join(arrContents)

In [ ]:
tempContent

In [ ]:
nouns = t.nouns(tempContent)

In [ ]:
count = Counter(nouns)
count

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict(count, orient='index').reset_index()
df

In [ ]:
df.to_excel('sr_count.xlsx', sheet_name='sr')